# Parabola
Extra required packages:
- matplotlib

In this notebook we want to solve a simple example where we approximate a parabola.

We consider the following optimization problem in two variables:

$$
\begin{aligned}
&\max x\\
&\text{s.t.}\\
&y \ge x^2,\\
&y \le - x^2 + 1.\\
\end{aligned}
$$

This model could be solved directly by Gurobi (or by hand) but here we want to solve it by approximating the function $y = x^2$ with a neural network.

### Geometrical solution

First, use matplotlib to plot the feasible region of the problem and find the optimal solution of the problem.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
x = np.arange(-1, 1, 0.005)
z = (x) ** 2

plt.plot(x, z)
plt.plot(x, -z + 1)
plt.show()

The point that maximize $x$ is the left most point above the blue parabola and below the orange one.
It is the point at the intersection of the two parabola with coordinates $\left(\frac{1}{\sqrt 2}, \frac{1}{2}\right)$.

Note that we only really need to approximate the parabola between $-1$ and $1$, we will restrict ourselves to this domain.

### Approximation with a neural network

In this notebook we approximate the parabola using a neural network with the MLPRegressor from scikit-learn.

For such a trivial example, we don't do any cross validation and will just visually verify that the approximation looks ok.

In [ ]:
# Import the MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

For our training set, we sample points uniformly in the interval $[-1, 1]$.

In [ ]:
x = np.arange(-1, 1, 0.005)
y = (x) ** 2

# Rearange points for scikit-learn
X = np.concatenate([x.ravel().reshape(-1, 1)], axis=1)
Y = y.ravel()

We can now run our regression.

Even though the function is simple, we still need a network of decent size.
We use 2 hidden layers of 56 neurons.

In [ ]:
# Run our regression
regression = make_pipeline(PolynomialFeatures(), LinearRegression())
regression.fit(X=X, y=Y)

We can now plot the approximation of the feasible region with the neural network.

In [ ]:
plt.plot(X, regression.predict(X))
plt.plot(X, -regression.predict(X) + 1)
plt.show()

The approximation is coarse but looks decent.

### Construct the optimization model

We will approximate the function $y = x^2$ with a neural network

Then we need to reformulate as
$$
\begin{aligned}
&\max x\\
&\text{s.t.}\\
&z = g(x) \approx  x^2\\
&y \geq z\\
&y \leq -z + 1\\
&- 1 \leq x \leq 1\\
\end{aligned}
$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from gurobi_ml import add_predictor_constr

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addVar(lb=-1, ub=1, name="x")
z = m.addVar(lb=-GRB.INFINITY, name="z")
y = m.addVar(lb=-GRB.INFINITY, name="y")
m.setObjective(x, gp.GRB.MAXIMIZE)

# Add constraint to predict value of y using kwnown and to compute features
pconstr = add_predictor_constr(m, regression, x, z)
# m.addConstr(z == x**2)
m.addConstr(y >= z)
m.addConstr(y <= -z + 1)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 10
m.Params.MIPGap = 0.01
m.Params.NonConvex = 2

In [ ]:
m.optimize()

In [ ]:
print(f"Maximal error in predicted values in solution {np.max(pconstr.get_error())}")

### Look at the solution

In [ ]:
x.X

In [ ]:
from math import sqrt

In [ ]:
x.X - 1 / sqrt(2)

In [ ]:
y.X

In [ ]:
z.X

Copyright © 2023 Gurobi Optimization, LLC